# Ratios - Growth

In [17]:
# Import necessary packages
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets

In [2]:
# Read nifty 50 index data
df = pd.read_csv('./data/annual_reports/nifty_50.csv')

In [3]:
df.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,EQ,INE012A01025
1,Adani Enterprises Ltd.,METALS,ADANIENT,EQ,INE423A01024
2,Adani Green Energy Ltd.,POWER,ADANIGREEN,EQ,INE364U01010
3,Adani Transmission Ltd.,POWER,ADANITRANS,BE,INE931S01010
4,Ambuja Cements Ltd.,CEMENT & CEMENT PRODUCTS,AMBUJACEM,EQ,INE079A01024


In [4]:
# Number of companies by industry sectors in nifty 50.
df['Industry'].value_counts()

FINANCIAL SERVICES          12
CONSUMER GOODS               8
PHARMA                       6
METALS                       5
CONSUMER SERVICES            3
OIL & GAS                    3
CEMENT & CEMENT PRODUCTS     2
POWER                        2
SERVICES                     1
CHEMICALS                    1
FERTILISERS & PESTICIDES     1
IT                           1
AUTOMOBILE                   1
TELECOM                      1
CONSTRUCTION                 1
HEALTHCARE SERVICES          1
INDUSTRIAL MANUFACTURING     1
Name: Industry, dtype: int64

Nifty 50 index is a heavy investor in finance, followed by consumer goods and pharmaceuticals. These 3 sectors contribute a little more than half of the campnies in the index.

## Year-to-year Growth Rate

Year-to-year growth rate is the rate of change in the specific metric of interest this year with respect to previous year. Two examples will be considered for analysis - total revenue and operating income.

### Example: Total Revenue

In [5]:
yoy_df = pd.DataFrame(columns=["Industry", "2021", "2020", "2019"])
years = [2021, 2020, 2019]

In [6]:
# Calculating year-to-year growth for companies in nifty 50.
for index, row in df.iterrows():
    ticker = row["Symbol"]
    income_stmt_path = f"./data/annual_reports/{ticker}/income_stmt.csv"
    is_df = pd.read_csv(income_stmt_path, index_col=[0])
    is_df_cols = [col.split("-")[0] for col in is_df.columns]
    is_df.columns = is_df_cols
    revenue = is_df.loc["Total Revenue"]
    for _, year in enumerate(years):
        if f"{year}" in revenue.index:
            yoy_df.at[str(ticker), str(year)] = (revenue[f"{year}"] - revenue[f"{year-1}"])*100/revenue[f"{year-1}"]
    yoy_df.at[str(ticker), "Industry"] = row["Industry"]
    yoy_df.at[str(ticker), "Company Name"] = row["Company Name"]

In [7]:
yoy_df.head()

,Industry,2021,2020,2019,Company Name
ACC,CEMENT & CEMENT PRODUCTS,NaN,-11.953147,5.782678,ACC Ltd.
ADANIENT,METALS,-8.905995,7.488857,12.400484,Adani Enterprises Ltd.
ADANIGREEN,POWER,22.705882,22.770896,40.313752,Adani Green Energy Ltd.
ADANITRANS,POWER,-13.048662,56.266349,85.206922,Adani Transmission Ltd.
AMBUJACEM,CEMENT & CEMENT PRODUCTS,NaN,-9.546277,4.080536,Ambuja Cements Ltd.


In [8]:
# Determine top 5 companies with highest average year-to-year growth in last 3 years.
temp_yoy_df = yoy_df.copy()
temp_yoy_df["2021-2020"] = temp_yoy_df["2021"] - temp_yoy_df["2020"]
temp_yoy_df["2020-2019"] = temp_yoy_df["2020"] - temp_yoy_df["2019"]
temp_yoy_df["yoy_mean"] = (temp_yoy_df["2021-2020"] + temp_yoy_df["2020-2019"])/2
temp_yoy_df.dropna(axis=0, inplace=True)
temp_yoy_df.sort_values("yoy_mean", ascending=False, inplace=True)

In [9]:
temp_yoy_df[["Company Name", "Industry", "yoy_mean"]].head()

,Company Name,Industry,yoy_mean
ICICIPRULI,ICICI Prudential Life Insurance Company Ltd.,FINANCIAL SERVICES,135.8033
INDUSTOWER,Indus Towers Ltd.,TELECOM,51.924432
SIEMENS,Siemens Ltd.,INDUSTRIAL MANUFACTURING,17.435053
NMDC,NMDC Ltd.,METALS,13.373396
PNB,Punjab National Bank,FINANCIAL SERVICES,12.006564


The above are the top 5 companies in Nifty 50 index with highest average year-to-year growth in the last 3 years. ICICI prudential being the top contender with 135% average year-to-year growth in last 3 years.

In [11]:
sector_yoy = temp_yoy_df.groupby(['Industry']).mean()
sector_yoy

/var/folders/zl/bqtzmz3s0xl5_ddgbjsqxsww0000gn/T/ipykernel_34989/1483552614.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  sector_yoy = temp_yoy_df.groupby(['Industry']).mean()


,2021,2020,2019,2021-2020,2020-2019,yoy_mean
Industry,,,,,,
AUTOMOBILE,-0.287075,-19.621105,3.487628,19.334030,-23.108733,-1.887351
CHEMICALS,-0.024128,3.058932,16.444267,-3.083060,-13.385335,-8.234198
CONSTRUCTION,-10.993527,-27.292508,24.740576,16.298981,-52.033083,-17.867051
CONSUMER GOODS,7.791886,-0.042512,13.416183,7.834399,-13.458695,-2.812148
CONSUMER SERVICES,-11.071530,16.174625,22.529155,-27.246155,-6.354530,-16.800342
FERTILISERS & PESTICIDES,35.957226,18.501179,24.759563,17.456046,-6.258383,5.598832
FINANCIAL SERVICES,26.025135,35.959907,21.045491,-9.934772,14.914417,2.489822
HEALTHCARE SERVICES,-6.106626,16.940993,16.668130,-23.047619,0.272863,-11.387378
INDUSTRIAL MANUFACTURING,37.125622,-23.979089,2.255516,61.104711,-26.234605,17.435053


The above table summarizes the year-to-year growth for each sector of the companies present in nifty 50.

In [16]:
sector_yoy.sort_values("yoy_mean", ascending=False, inplace=True)
sector_yoy.head()

,2021,2020,2019,2021-2020,2020-2019,yoy_mean
Industry,,,,,,
TELECOM,106.944980,-1.218833,3.096116,108.163813,-4.314949,51.924432
INDUSTRIAL MANUFACTURING,37.125622,-23.979089,2.255516,61.104711,-26.234605,17.435053
FERTILISERS & PESTICIDES,35.957226,18.501179,24.759563,17.456046,-6.258383,5.598832
FINANCIAL SERVICES,26.025135,35.959907,21.045491,-9.934772,14.914417,2.489822
METALS,13.263016,-6.967148,15.431101,20.230164,-22.398249,-1.084042


Telecom sector experienced the largest average year-to-year growth in the nifty 50 index. It is followed by industrial manufacturing and fertilisers & pesticides. Note that only 4 sectors reported postive average year-to-year growth, while rest all the sectors reported negative growth.

In [ ]:
# Plot the sector-wise year-to-year growth
# WIP
sector = widgets.Dropdown(
    options = list(sector_yoy.index),
    value="TELECOM"
    description="Sector"
)